In [1]:
%autosave 0

Autosave disabled


## 数据集：科技、汽车、娱乐、军事、体育 5类
### 格式：\[[word1 word2 ... word] category]

In [2]:
import numpy as np
import pandas as pd
import jieba

In [3]:
df_technology = pd.read_csv('./data/technology_news.csv', encoding='utf-8')
df_technology = df_technology.dropna()

df_car = pd.read_csv('./data/car_news.csv', encoding='utf-8')
df_car = df_car.dropna()

df_entertainment = pd.read_csv('./data/entertainment_news.csv', encoding='utf-8')
df_entertainment = df_entertainment.dropna()

df_military = pd.read_csv('./data/military_news.csv', encoding='utf-8')
df_military = df_military.dropna()

df_sports = pd.read_csv('./data/sports_news.csv', encoding='utf-8')
df_sports = df_sports.dropna()

technology = df_technology.content.values.tolist()[1000:21000]
car = df_car.content.values.tolist()[1000:21000]
entertainment = df_entertainment.content.values.tolist()[:20000]
military = df_military.content.values.tolist()[:20000]
sports = df_sports.content.values.tolist()[:20000]

In [6]:
stopwords = pd.read_csv('./data/stopwords.txt', index_col=False, quoting=3, sep="\t", names=['stopword'], encoding='utf-8')
stopwords = stopwords['stopword'].values

In [7]:
def process_text(content_lines, sentences, category):
    for line in content_lines:
        try:
            segs = jieba.lcut(line)
            segs = filter(lambda x: len(x)>1, segs)
            segs = filter(lambda x: x not in stopwords, segs)
            sentences.append((" ".join(list(segs)), category))
        except:
            print(line)
            continue

In [8]:
sentences = []

process_text(technology, sentences, 'technology')
process_text(car, sentences, 'car')
process_text(entertainment, sentences, 'entertainment')
process_text(military, sentences, 'military')
process_text(sports, sentences, 'sports')

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.490 seconds.
Prefix dict has been built succesfully.


In [9]:
import random

In [10]:
random.shuffle(sentences)

In [11]:
for sentence in sentences[:10]:
    print(sentence[0], sentence[1])

规划 预计 2020 中国 汽车 年产量 3000 万辆 2025 3500 万辆 car
山东 菏泽 武县 伯乐 大街 显眼 手机 通讯 招牌 春节 张灯结彩 景象 交相 呼应 热情 导购员 介绍 OPPO 最新 机型 拥有 800 人口 乡镇 市场 计入 合作 网点 运营商 门店 OPPO 自营 体验 technology
提前 判断 航母 下水 标志 悬挂 满旗 大型 船舶 下水 时要 隆重 仪式 悬挂 满旗 满灯 军舰 特有 庆典 方式 庞大 舰艇 悬挂 满旗 提前 判别 航母 下水 最为 简单 直观 判断 military
投资 重头戏 融创 150 亿元 投资 分为 乐视 优质 业务 板块 第一 围绕 乐视 体系 上市公司 视网 展开 融创 收购 乐视 掌门人 贾跃亭 持有 8.61% 股权 围绕 以乐视 超级 电视 板块 乐视致 展开 融创 79.5 亿元 价格 增发 33.5% 股权 49.5 亿元 从乐 视网 和鑫乐 资产 手中 购买 共计 26.1% 增发 老股 增资 亿元 增发 10% 第三 围绕 乐视 影业 展开 整体 估值 亿元 融创 10.5 亿元 收购 乐视 控股 持有 15% 股份 technology
聚众 互动 制定 战队 赛制 强化 队长 战队 作用 意义 团队 记分牌 增加 全新 用途 每支 队伍 局势 实时 变化 选择 战术 达成 团队 目标 著名 导演 王岳伦 比赛 战队 队长 评论 团队 赛制 队员 感受 以往 德州 扑克 比赛 体会 不到 团队 凝聚力 这是 赛制 赛后 世界 著名 职业 扑克 选手 David Chiu 提到 比赛 赛制 展现出 团队 价值 团队 赛制 非常适合 中国 德州 扑克 中国 推广 起到 作用 sports
黄蓉 郭靖 扮演者 观众 心中 经典 形象 合二为一 见仁见智 范畴 entertainment
比巴方 方队 巴方 方队 地上 一道 一道 行进 方向线 实际上 方队 设计 方队 实际上 可行 把握 行进 方向 military
去年 夏窗 曼城 曼联 切尔西 阿森纳 引援 投入 跻身 欧洲 前十名 曼彻斯特 双雄 包揽 两位 曼城 耗资 2.13 欧元 唯一 投入 超过 欧元 俱乐部 投入 1.91 欧元 引进 斯通斯 萨内 赫苏斯 京多安 诺利托 布拉沃 大牌 新援 新援 表现 民

In [12]:
sentences[0]

('规划 预计 2020 中国 汽车 年产量 3000 万辆 2025 3500 万辆', 'car')

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
x, y = zip(*sentences)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1234)

len(x_train)

65696

In [15]:
from sklearn.feature_extraction.text import CountVectorizer

In [16]:
vec = CountVectorizer(analyzer='word',
                     max_features=4000,)
vec.fit(x_train)

def get_features(x):
    vec.transform(x)

In [17]:
from sklearn.naive_bayes import MultinomialNB

In [18]:
classifer = MultinomialNB()
classifer.fit(vec.transform(x_train), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [19]:
classifer.score(vec.transform(x_test), y_test)

0.8378921411936618

In [20]:
len(x_test)

21899

In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

vec = CountVectorizer(analyzer='word',
                     ngram_range=(1,4),
                     max_features=20000,)
vec.fit(x_train)

def get_features(x):
    vec.transform(x)
    
classifer = MultinomialNB()
classifer.fit(vec.transform(x_train), y_train)

classifer.score(vec.transform(x_test), y_test)

0.8802685054112059

In [26]:
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score

In [30]:
def stratifiedkfold_cv(x, y, clf_class, shuffle=True, n_folds=5, **kwargs):
    stratifiedk_fold = StratifiedKFold(y, n_folds=n_folds, shuffle=shuffle)
    y_pred = y[:]
    for train_index, test_index in stratifiedk_fold:
        X_train, X_test = x[train_index], x[test_index]
        y_train = y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train, y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred

NB = MultinomialNB
print(precision_score(y, stratifiedkfold_cv(vec.transform(x), np.array(y), NB), average='macro'))

0.8810618069116188


In [34]:
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB

class TextClassifier():

    def __init__(self, classifier=MultinomialNB()):
        self.classifier = classifier
        self.vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,4), max_features=20000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)
    
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict('这 是 有史以来 最 大 的 一 次 军舰 演习'))
print(text_classifier.score(x_test, y_test))

['military']
0.8802685054112059


In [39]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

class TextClassifier():
    def __init__(self, classifier=SVC(kernel='linear')):
        self.classifier = classifier
        self.vectorizer = TfidfVectorizer(analyzer='word',
                                        ngram_range=(1,4),
                                        max_features=12000)
        
    def features(self, X):
        return self.vectorizer.transform(X)
    
    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)
    
    def predict(self, x):
        return self.classifier.predict(self.features([x]))
    
    def score(self, X, y):
        return self.classifier.score(self.features(X), y)
    
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict('这 是 有史以来 最 大 的 一 次 军舰 演习'))
print(text_classifier.score(x_test, y_test))

['military']
0.8762043928946527
